In [33]:
import os
import dotenv
import sqlalchemy
import pandas as pd
import numpy as np
import math as m

dotenv.load_dotenv(override=True, dotenv_path ='.env')

username="studente_dapt"
password="Ep1c0d3!!D4t4**4n4lys1s"
host="epicode-data-pt-mysql.cvetyjye2qbl.eu-central-1.rds.amazonaws.com"
dbname="AdventureWorksDW"


connection_string = "mysql+pymysql://"+username+":"+password+"@"+host+"/"+dbname
db_engine = sqlalchemy.create_engine(connection_string)

### Esercizio 1

Estraiamo la tabella dimproduct dal database AdventureWorks, e valutiamo quanto segue:
Quanti dati ci sono in totale? 
* Quali sono i metadati? Le etichette di colonna, la tipologia di dati in esso contenuto, il formato e la quantità. Con il metodo info() è possibile sapere anche quanta memoria occupa il database  
* Stampiamo il primo elemento 
* Stampiamo l'ultimo elemento 
* Riusciamo a stampare cinque elementi a caso? 
* Quali sono i colori disponibili?

In [30]:
query = "SELECT * FROM dimproduct" 
df = pd.read_sql(query, db_engine)
#Quali sono i metadati?
df_shape = df.shape # restituisce righe e colonne (attributo)
print(df_shape)
df_index = df.index # restituisce le etichette delle righe
print(df_index)
df_columns = df.columns # restituisce le etichette delle colonne
print(df_columns)
df_dtype = df.dtypes
print(df_dtype)
df_info = df.info()
print(df_info)
print(df.describe())
print(df.head(1))
print(df.tail(1))
print(df.sample(5))
print(df['Color'].unique())

(606, 36)
RangeIndex(start=0, stop=606, step=1)
Index(['ProductKey', 'ProductAlternateKey', 'ProductSubcategoryKey',
       'WeightUnitMeasureCode', 'SizeUnitMeasureCode', 'EnglishProductName',
       'SpanishProductName', 'FrenchProductName', 'StandardCost',
       'FinishedGoodsFlag', 'Color', 'SafetyStockLevel', 'ReorderPoint',
       'ListPrice', 'Size', 'SizeRange', 'Weight', 'DaysToManufacture',
       'ProductLine', 'DealerPrice', 'Class', 'Style', 'ModelName',
       'LargePhoto', 'EnglishDescription', 'FrenchDescription',
       'ChineseDescription', 'ArabicDescription', 'HebrewDescription',
       'ThaiDescription', 'GermanDescription', 'JapaneseDescription',
       'TurkishDescription', 'StartDate', 'EndDate', 'Status'],
      dtype='object')
ProductKey                        int64
ProductAlternateKey              object
ProductSubcategoryKey           float64
WeightUnitMeasureCode            object
SizeUnitMeasureCode              object
EnglishProductName               obj

### 2

* In media quanto pesano i prodotti (per questo esercizio, ignoriamo l'unità di misura, ma usiamo solo i valori della colonna relativa)?
* Quanto pesa il più leggero?
* Quanto pesa il più pesante?
* Quanti prodotti pesano più di 100?
* Quanto costano in media i prodotti (colonna DealerPrice)?
* Quali sono i quartili dei prezzi?

In [3]:
average_weight = np.mean(df['Weight']) # or df['Weight'].mean()
print('The average weight is: ', average_weight)

max_weight = np.max(df['Weight'])
print('The max weight is: ', max_weight)

min_weight = np.min(df['Weight'])
print('The min weight is: ', min_weight)

filter_more_than_100 = df['Weight'] > 100
print(filter_more_than_100.sum()) # funziona perché i true valgono 1 e i false valgono 0
var = df.loc[filter_more_than_100]
print(var.shape)

average_price = np.mean(df['DealerPrice'])
print('The average price is: ', average_price)
print(df.describe())

The average weight is:  56.1932269503546
The max weight is:  1050.0
The min weight is:  2.12
28
(28, 36)
The average price is:  448.5970597468353
       ProductKey  ProductSubcategoryKey  StandardCost  FinishedGoodsFlag  \
count   606.00000             397.000000    397.000000         606.000000   
mean    303.50000              12.128463    436.823073           0.655116   
min       1.00000               1.000000      0.860000           0.000000   
25%     152.25000               2.000000     37.120000           0.000000   
50%     303.50000              13.000000    204.630000           1.000000   
75%     454.75000              16.000000    660.910000           1.000000   
max     606.00000              37.000000   2171.290000           1.000000   
std     175.08141               9.423328    497.343079           0.475723   

       SafetyStockLevel  ReorderPoint    ListPrice       Weight  \
count        606.000000    606.000000   395.000000   282.000000   
mean         495.201320   

### 3

* Qual è il prezzo medio per i prodotti di colore blu?
* Qual è il prezzo medio per i prodotti di colore rosso o nero?
* Qual è il prezzo massimo per i prodotti di taglia 42 e peso oltre i 10 Kg?
* Qual è il nome inglese e il costo di produzione (StandardCost) di tutti i prodotti di taglia 42, peso oltre i 10 Kg e colore argento?
* Visualizziamo lo StandardCost e il DealerPrice degli ultimi 20 elementi del dataset: quali sono le differenze? C'è un pattern? C'è qualche elemento che non lo segue?

In [4]:
# filtri

filter_blue = df.loc[:, 'Color'] == 'Blue'
filter_black = df.loc[:, 'Color'] == 'Black'
filter_red = df.loc[:, 'Color'] == 'Red'
filter_silver = df.loc[:, 'Color'] == 'Silver'
filter_bor = filter_black | filter_red
filter_42 = df.loc[:, 'Size'] == '42'
filter_weight = df.loc[:, 'Weight'] > 10
filter_size_weight_silver = filter_42 & filter_weight & filter_silver
average_price = np.mean(df['DealerPrice'])

In [5]:
df_blue = df.loc[filter_blue]
avg_blue_price = df_blue.loc[:, 'DealerPrice'].mean()
print(avg_blue_price)

df_bor = df.loc[filter_bor]
avg_bor_price = df_bor.loc[:, 'DealerPrice'].mean()
print(avg_bor_price)

df_sws = df.loc[filter_size_weight_silver]
avg_sws_price = df_sws.loc[:, 'DealerPrice'].mean()
print(avg_sws_price)

df_tail = df.tail(20)
var = df_tail.loc[:, ['StandardCost', 'DealerPrice']]
print(var)

516.0631785714286
550.2481873684211
1095.10556
     StandardCost  DealerPrice
586        419.78      461.694
587        419.78      461.694
588        419.78      461.694
589        419.78      461.694
590        308.22      338.994
591        308.22      338.994
592        308.22      338.994
593        308.22      338.994
594        308.22      338.994
595        294.58      323.994
596        294.58      323.994
597        294.58      323.994
598        294.58      323.994
599        294.58      323.994
600         23.97       32.394
601         44.95       60.744
602         53.94       72.894
603        343.65      323.994
604        343.65      323.994
605        343.65      323.994


### 4

selezioniamo amazon.csv, un dataset contenente una serie di recensioni su Amazon. • Valutiamo la dimensione del dataset • Visualizziamo dieci righe a caso; • Osserviamo quali sono i nomi di colonna; • Il dataset è bilanciato, ovvero, il numero di recensioni positive è uguale a quello delle negative, oppure no?

In [6]:
import pandas as pd

amazon = pd.read_csv('amazon.csv') # importo il dataset
print(amazon.shape, 
amazon.sample(10), 
amazon.columns, 
amazon.value_counts('Positive'))

(20000, 2)                                               reviewText  Positive
2873   This is a pretty robust app that can be played...         1
6173   works fine for me and its a pretty good app. w...         1
10887  I love this app. I use it all the time. Works ...         1
16101  I've gotten lots 10 of these types of apps alr...         0
12169  clock works great, is nice and loud to take yo...         1
10760  The app didn't run correctly on the Kindle Fir...         0
4452   This is one of the best free puzzle apps I hav...         1
13584  I love this game! It is fun and very well made...         1
9591   This app has come in handy several times and h...         1
2938   We've been playing all day.  Love it!  Great o...         1 Index(['reviewText', 'Positive'], dtype='object') Positive
1    15233
0     4767
Name: count, dtype: int64


### 5 

Il dataset diabetes.csv raccoglie persone con diabete o meno, e il valore di diverse variabili fisiologiche dei pazienti. • Osserviamone le dimensioni e un'anteprima di cinque righe; • Prendiamoci un po' di tempo per dare un'occhiata ai metadati delle colonne; • Stampiamo dei descrittori statistici del dataset; • Selezioniamo i dati relativi a diverse fasce di età: 20-29, 30-39, 40-50; • Qual è la media della pressione sanguigna diastolica per le diverse fasce di età?

In [7]:
import pandas as pd
diabetes_df = pd.read_csv('diabetes.csv')
# dimensioni e anteprima 5 righe
print(diabetes_df.shape)
      #diabetes_df.head(5))
# metadati
print(diabetes_df.columns)
print(diabetes_df.index)


# descrittori statistici
print(diabetes_df.info())
print(diabetes_df.describe())

(768, 9)
Index(['Number of times pregnant',
       'Plasma glucose concentration a 2 hours in an oral glucose tolerance test',
       'Diastolic blood pressure (mm Hg)', 'Triceps skin fold thickness (mm)',
       '2-Hour serum insulin (mu U/ml)',
       'Body mass index (weight in kg/(height in m)^2)',
       'Diabetes pedigree function', 'Age (years)', 'Class variable'],
      dtype='object')
RangeIndex(start=0, stop=768, step=1)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   Number of times pregnant                                                  768 non-null    int64  
 1   Plasma glucose concentration a 2 hours in an oral glucose tolerance test  768 non-null    int64  
 2   Diastolic blood pressure (mm Hg)            

In [8]:
filtro20 = diabetes_df.loc[:, 'Age (years)'] > 20
filtro29 = diabetes_df.loc[:, 'Age (years)'] <= 29

filtro20_29 = filtro20 & filtro29

filtro30 = diabetes_df.loc[:, 'Age (years)'] > 30
filtro39 = diabetes_df.loc[:, 'Age (years)'] <= 39

filtro30_39 = filtro30 & filtro39

filtro40 = diabetes_df.loc[:, 'Age (years)'] > 40
filtro50 = diabetes_df.loc[:, 'Age (years)'] <= 50

filtro40_50 = filtro40 & filtro50

In [2]:
diabetes_df['Diastolic blood pressure (mm Hg)'].loc[filtro20_29].mean()

NameError: name 'diabetes_df' is not defined

In [10]:
diabetes_df['Diastolic blood pressure (mm Hg)'].loc[filtro30_39].mean()

70.36805555555556

In [11]:
diabetes_df['Diastolic blood pressure (mm Hg)'].loc[filtro40_50].mean()

74.78761061946902

### 6

Il dataset insurance.csv contiene dati rispetto a caratteristiche e abitudini delle persone, e della zona in cui vivono, rispetto ai costi individuali per le cure mediche come premio per le assicurazioni sulla salute. • Visualizziamone le dimensioni, un'anteprima, e osserviamo i nomi di colonna; • Quali sono le medie di charges rispetto a region? Ci sono differenze significative? • E rispetto a smoker? E a sex? • Quali sono i descrittori statistici di bmi? Quali sono minimo, media e massimo di charges rispetto ai diversi quartili dei valori di bmi?

In [12]:
insurance = pd.read_csv('insurance.csv')

In [13]:
# dimensioni, anteprima e nomi di colonna

insurance.shape

(1338, 7)

In [14]:
insurance.index

RangeIndex(start=0, stop=1338, step=1)

In [15]:
insurance.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [16]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [17]:
insurance.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [18]:
insurance.sample(5)

,age,sex,bmi,children,smoker,region,charges
391,19,female,37.430,0,no,northwest,2138.07070
824,60,male,24.320,0,no,northwest,12523.60480
231,59,female,27.830,3,no,southeast,14001.28670
850,37,female,30.780,0,yes,northeast,37270.15120
692,20,male,32.395,1,no,northwest,2362.22905


In [19]:
# filtro metodo 1

filter_ne = insurance.loc[:, 'region'] == 'northeast'
filter_nw = insurance.loc[:, 'region'] == 'northwest'
filter_se = insurance.loc[:, 'region'] == 'southeast'
filter_sw = insurance.loc[:, 'region'] == 'southwest'

print(insurance['charges'].loc[filter_ne].mean())
print(insurance['charges'].loc[filter_nw].mean())
print(insurance['charges'].loc[filter_se].mean())
print(insurance['charges'].loc[filter_sw].mean())


13406.384516385804
12417.57537396923
14735.411437609888
12346.937377292308


In [20]:
# filtro metodo 2

import numpy as np
region = np.array(insurance['region'].unique())

for i in region:
    filtro = insurance.loc[:, 'region'] == i
    media =  insurance['charges'].loc[filtro].mean()
    print('La regione ', i, ' ha come media ', media)


La regione  southwest  ha come media  12346.937377292308
La regione  southeast  ha come media  14735.411437609888
La regione  northwest  ha come media  12417.57537396923
La regione  northeast  ha come media  13406.384516385804


In [21]:
# differenze di sesso

import numpy as np

sex = np.array(insurance['sex'].unique())

for i in sex:
    filtro = insurance.loc[:, 'sex'] == i
    media =  insurance['charges'].loc[filtro].mean()
    print(' Il sesso ', i, ' paga ', media)

 Il sesso  female  paga  12569.578843835347
 Il sesso  male  paga  13956.751177721893


In [22]:
import numpy as np

smoker = np.array(insurance['smoker'].unique())

for i in smoker:
    filtro = insurance.loc[:, 'smoker'] == i
    media =  insurance['charges'].loc[filtro].mean()
    print( i, media) 

yes 32050.23183153284
no 8434.268297856204


Il dataset pokemon.csv contiene un database di Pokémon, con dati quali nome, tipi di attacco, valori di attacco/difesa/eccetera, e se sono o meno leggendari. • Verifichiamo la dimensione, un'anteprima e osserviamo i nomi di colonna; • È verosimile che la prima colonna dovrebbe essere un indice? • Confrontiamolo con l'indice messo automaticamente da Pandas: combaciano? • Se no, settare la prima colonna come indice.

In [23]:
import pandas as pd

pokemon = pd.read_csv('pokemon.csv')

pokemon.shape

In [24]:
pokemon.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [25]:
pokemon.head(10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False


In [26]:
pokemon.set_index('#')

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True


Quali sono i Pokémon leggendari? • E quali sono i leggendari di tipo 1 Grass? • E leggendari di tipo 1 Ice o Fire? • Trasformiamo la colonna Name nell'indice; • Quali sono i Pokémon della prima generazione con attacco > 50 e HP < 60?

In [34]:

for i in pokemon:
    pokemon_leg = []
    if 'Legendary' == True:
        pokemon_leg.append(i)

In [35]:
pokemon_leg

[]